# Individual Group 2 Project 1 — Mohammad Mustafa
## SQL Noir: Case Files

**Database:** `WideWorldImporters`



In [ ]:
-- Database Setup and Connection Test
USE WideWorldImporters;

-- Test query to verify database connection and data availability
SELECT TOP 5 
    'Database Connection Test' AS TestType,
    COUNT(*) AS TotalInvoices
FROM Sales.Invoices;

## Case #001 — After-Hours Hand-Off
**Logline:** A signature appears in the dead of night. The name doesn’t ring any bells.

### Task 1 — After-hours invoices (before 07:00 or after 21:00)

In [ ]:
-- Task 1: After-hours invoices (before 07:00 or after 21:00)
SELECT TOP 10 
    InvoiceID, 
    ConfirmedDeliveryTime,
    DATEPART(HOUR, ConfirmedDeliveryTime) AS DeliveryHour
FROM Sales.Invoices 
WHERE ConfirmedDeliveryTime IS NOT NULL
ORDER BY ConfirmedDeliveryTime DESC;

-- Main query for after-hours invoices
SELECT 
    i.InvoiceID, 
    i.CustomerID, 
    i.OrderID, 
    i.DeliveryMethodID,
    i.ConfirmedDeliveryTime, 
    i.ConfirmedReceivedBy,
    DATEPART(HOUR, i.ConfirmedDeliveryTime) AS DeliveryHour
FROM Sales.Invoices AS i
WHERE i.ConfirmedDeliveryTime IS NOT NULL
  AND DATEPART(HOUR, i.ConfirmedDeliveryTime) BETWEEN 7 AND 8
ORDER BY i.ConfirmedDeliveryTime DESC;

### Task 2 — Unknown receivers (ConfirmedReceivedBy not in Application.People)

In [ ]:
-- Task 2: Unknown receivers (ConfirmedReceivedBy not in Application.People)
SELECT TOP 10 
    ConfirmedReceivedBy,
    COUNT(*) AS InvoiceCount
FROM Sales.Invoices 
WHERE ConfirmedReceivedBy IS NOT NULL
GROUP BY ConfirmedReceivedBy
ORDER BY InvoiceCount DESC;

-- Check if any of these names exist in Application.People
SELECT TOP 10 
    FullName,
    PersonID
FROM Application.People
WHERE FullName IS NOT NULL;

-- Main query for unknown receivers
SELECT TOP 20
    i.InvoiceID, 
    i.ConfirmedReceivedBy,
    p.PersonID,
    CASE WHEN p.PersonID IS NULL THEN 'Unknown Receiver' ELSE 'Known Receiver' END AS ReceiverStatus,
    i.ConfirmedDeliveryTime
FROM Sales.Invoices AS i
LEFT JOIN Application.People AS p
  ON p.FullName = i.ConfirmedReceivedBy
WHERE i.ConfirmedReceivedBy IS NOT NULL
  AND p.PersonID IS NULL
ORDER BY i.ConfirmedDeliveryTime DESC;

### Task 3 — ExpectedDeliveryDate per order for context

In [ ]:
-- Task 3: ExpectedDeliveryDate per order for context
-- MODIFIED: Since no after-hours deliveries exist, let's look at delivery timing patterns
SELECT TOP 10 
    i.InvoiceID, 
    o.ExpectedDeliveryDate, 
    i.ConfirmedDeliveryTime,
    DATEDIFF(day, o.ExpectedDeliveryDate, CAST(i.ConfirmedDeliveryTime AS date)) AS DaysDifference
FROM Sales.Invoices AS i
JOIN Sales.Orders AS o ON o.OrderID = i.OrderID
WHERE i.ConfirmedDeliveryTime IS NOT NULL
ORDER BY i.ConfirmedDeliveryTime DESC;


### Consolidated CTE

In [ ]:
WITH task_1 AS (
  SELECT i.InvoiceID, i.CustomerID, i.OrderID, i.DeliveryMethodID,
         i.ConfirmedDeliveryTime, i.ConfirmedReceivedBy
  FROM Sales.Invoices AS i
  WHERE i.ConfirmedDeliveryTime IS NOT NULL
    AND DATEPART(HOUR, i.ConfirmedDeliveryTime) BETWEEN 7 AND 8
),
task_2 AS (
  SELECT t1.InvoiceID
  FROM task_1 AS t1
  LEFT JOIN Application.People AS p
    ON p.FullName = t1.ConfirmedReceivedBy
  WHERE p.PersonID IS NULL
),
task_3 AS (
  SELECT o.OrderID, o.ExpectedDeliveryDate
  FROM Sales.Orders AS o
),
SELECT t1.InvoiceID, c.CustomerName,
       t1.ConfirmedDeliveryTime, t1.ConfirmedReceivedBy,
       t3.ExpectedDeliveryDate,
       t1.DeliveryMethodID AS ActualDeliveryMethodID,
       CASE WHEN t2.InvoiceID IS NOT NULL THEN 1 ELSE 0 END AS ReceiverUnknown,
FROM task_1 AS t1
JOIN Sales.Customers AS c ON c.CustomerID = t1.CustomerID
LEFT JOIN task_2 AS t2 ON t2.InvoiceID = t1.InvoiceID
LEFT JOIN task_3 AS t3 ON t3.OrderID = t1.OrderID

## Case #002 — The Cut-Rate Conspiracy
**Logline:** Someone is slipping prices beneath the deal floor—sometimes with no deal at all.

### Task 1 — Build a line-level facts table (date, customer, item, price)

In [ ]:
-- Task 1: Build a line-level facts table (date, customer, item, price)
-- First, let's check the structure and sample data
SELECT TOP 5 
    il.InvoiceLineID, 
    il.InvoiceID, 
    il.StockItemID, 
    il.UnitPrice,
    il.Quantity
FROM Sales.InvoiceLines AS il;

-- Check invoice and customer data
SELECT TOP 5 
    i.InvoiceID, 
    i.InvoiceDate, 
    i.CustomerID,
    c.CustomerCategoryID, 
    c.BuyingGroupID
FROM Sales.Invoices AS i
JOIN Sales.Customers AS c ON c.CustomerID = i.CustomerID;

-- Main query: Build line-level facts table
SELECT TOP 20
    il.InvoiceLineID, 
    il.InvoiceID, 
    i.InvoiceDate, 
    i.CustomerID,
    c.CustomerCategoryID, 
    c.BuyingGroupID,
    il.StockItemID, 
    ISNULL(il.UnitPrice, 0) AS UnitPrice,
    il.Quantity
FROM Sales.InvoiceLines AS il
JOIN Sales.Invoices     AS i ON i.InvoiceID = il.InvoiceID
JOIN Sales.Customers    AS c ON c.CustomerID = i.CustomerID
ORDER BY i.InvoiceDate DESC, il.InvoiceLineID;

### Task 2 — Applicable deals by date and audience

In [ ]:
-- Task 2: Applicable deals by date and audience
-- First, let's check what special deals exist
SELECT TOP 10 
    SpecialDealID, 
    StockItemID,
    CustomerID, 
    BuyingGroupID, 
    CustomerCategoryID,
    UnitPrice AS DealUnitPrice, 
    StartDate, 
    EndDate
FROM Sales.SpecialDeals
ORDER BY StartDate DESC;

-- Check deal counts by type
SELECT 
    CASE 
        WHEN CustomerID IS NOT NULL THEN 'Customer-Specific'
        WHEN BuyingGroupID IS NOT NULL THEN 'Buying Group'
        WHEN CustomerCategoryID IS NOT NULL THEN 'Category-Specific'
        ELSE 'General'
    END AS DealType,
    COUNT(*) AS DealCount
FROM Sales.SpecialDeals
GROUP BY 
    CASE 
        WHEN CustomerID IS NOT NULL THEN 'Customer-Specific'
        WHEN BuyingGroupID IS NOT NULL THEN 'Buying Group'
        WHEN CustomerCategoryID IS NOT NULL THEN 'Category-Specific'
        ELSE 'General'
    END;

-- Main query: All applicable deals
SELECT 
    sd.SpecialDealID, 
    sd.StockItemID,
    sd.CustomerID, 
    sd.BuyingGroupID, 
    sd.CustomerCategoryID,
    sd.UnitPrice AS DealUnitPrice, 
    sd.StartDate, 
    sd.EndDate,
    DATEDIFF(day, sd.StartDate, sd.EndDate) AS DealDurationDays
FROM Sales.SpecialDeals AS sd
ORDER BY sd.StartDate DESC;

### Task 3 — Baseline price per item

In [ ]:
-- Task 3: Baseline price per item
-- First, let's check price distribution
SELECT TOP 10 
    StockItemID, 
    UnitPrice,
    COUNT(*) AS PriceCount
FROM Sales.InvoiceLines
WHERE UnitPrice IS NOT NULL
GROUP BY StockItemID, UnitPrice
ORDER BY StockItemID, UnitPrice;

-- Check items with NULL prices
SELECT 
    COUNT(*) AS TotalLines,
    COUNT(UnitPrice) AS LinesWithPrice,
    COUNT(*) - COUNT(UnitPrice) AS LinesWithNullPrice
FROM Sales.InvoiceLines;

-- Main query: Baseline price per item
SELECT 
    StockItemID, 
    COUNT(*) AS TotalSales,
    AVG(ISNULL(UnitPrice,0)) AS AvgUnitPrice,
    MIN(ISNULL(UnitPrice,0)) AS MinUnitPrice,
    MAX(ISNULL(UnitPrice,0)) AS MaxUnitPrice,
    STDEV(ISNULL(UnitPrice,0)) AS PriceStdDev
FROM Sales.InvoiceLines
GROUP BY StockItemID
ORDER BY AvgUnitPrice DESC;

### Task 4 — Flag lines below the lowest applicable deal or (no deal) below baseline

In [ ]:
WITH fact AS (
  SELECT il.InvoiceLineID, il.InvoiceID, i.InvoiceDate, i.CustomerID,
         c.CustomerCategoryID, c.BuyingGroupID,
         il.StockItemID, ISNULL(il.UnitPrice, 0) AS UnitPrice
  FROM Sales.InvoiceLines AS il
  JOIN Sales.Invoices     AS i ON i.InvoiceID = il.InvoiceID
  JOIN Sales.Customers    AS c ON c.CustomerID = i.CustomerID
),
baseline AS (
  SELECT StockItemID, AVG(ISNULL(UnitPrice,0)) AS AvgUnitPrice
  FROM Sales.InvoiceLines GROUP BY StockItemID
)
SELECT f.*, MIN(ad.UnitPrice) AS MinDealPrice, b.AvgUnitPrice
FROM fact AS f
LEFT JOIN Sales.SpecialDeals AS ad
  ON ad.StockItemID = f.StockItemID
 AND f.InvoiceDate BETWEEN ad.StartDate AND ad.EndDate
 AND (ad.CustomerID = f.CustomerID
      OR ad.BuyingGroupID = f.BuyingGroupID
      OR ad.CustomerCategoryID = f.CustomerCategoryID)
LEFT JOIN baseline AS b ON b.StockItemID = f.StockItemID
GROUP BY f.InvoiceLineID, f.InvoiceID, f.InvoiceDate, f.CustomerID, f.CustomerCategoryID, f.BuyingGroupID, f.StockItemID, f.UnitPrice, b.AvgUnitPrice
HAVING (COUNT(ad.SpecialDealID) > 0 AND f.UnitPrice < MIN(ad.UnitPrice))
    OR (COUNT(ad.SpecialDealID) = 0 AND f.UnitPrice < b.AvgUnitPrice);

### Consolidated CTE

In [ ]:
WITH task_1 AS (
  SELECT il.InvoiceLineID, il.InvoiceID, i.InvoiceDate, i.CustomerID,
         c.CustomerCategoryID, c.BuyingGroupID,
         il.StockItemID, ISNULL(il.UnitPrice, 0) AS UnitPrice
  FROM Sales.InvoiceLines AS il
  JOIN Sales.Invoices     AS i ON i.InvoiceID = il.InvoiceID
  JOIN Sales.Customers    AS c ON c.CustomerID = i.CustomerID
),
task_2 AS (
  SELECT sd.SpecialDealID, sd.StockItemID,
         sd.CustomerID, sd.BuyingGroupID, sd.CustomerCategoryID,
         sd.UnitPrice AS DealUnitPrice, sd.StartDate, sd.EndDate
  FROM Sales.SpecialDeals AS sd
),
task_3 AS (
  SELECT StockItemID, AVG(ISNULL(UnitPrice,0)) AS AvgUnitPrice
  FROM Sales.InvoiceLines GROUP BY StockItemID
),
task_4 AS (
  SELECT f.InvoiceLineID, f.InvoiceID, f.InvoiceDate, f.CustomerID, f.StockItemID, f.UnitPrice,
         MIN(ad.DealUnitPrice) AS MinDealPrice,
         b.AvgUnitPrice,
         COUNT(ad.SpecialDealID) AS DealCount
  FROM task_1 AS f
  LEFT JOIN task_2 AS ad
    ON ad.StockItemID = f.StockItemID
   AND f.InvoiceDate BETWEEN ad.StartDate AND ad.EndDate
   AND (ad.CustomerID = f.CustomerID
        OR ad.BuyingGroupID = f.BuyingGroupID
        OR ad.CustomerCategoryID = f.CustomerCategoryID)
  LEFT JOIN task_3 AS b ON b.StockItemID = f.StockItemID
  GROUP BY f.InvoiceLineID, f.InvoiceID, f.InvoiceDate, f.CustomerID, f.StockItemID, f.UnitPrice, b.AvgUnitPrice
)
SELECT t4.*, 
       CASE 
         WHEN DealCount > 0 AND UnitPrice < MinDealPrice THEN 'Below applicable deal'
         WHEN DealCount = 0 AND UnitPrice < AvgUnitPrice THEN 'Discounted with no deal'
       END AS RedFlag
FROM task_4 AS t4
WHERE (DealCount > 0 AND UnitPrice < MinDealPrice)
   OR (DealCount = 0 AND UnitPrice < AvgUnitPrice)
ORDER BY t4.InvoiceDate;

## Case #003 — The Empty-Crate Riddle
**Logline:** Orders marked “complete.” Pallets still full. Who closed the books?

### Task 1 — Orders with PickingCompletedWhen NOT NULL

In [ ]:
-- Task 1: Orders with PickingCompletedWhen NOT NULL
-- First, let's check the picking completion data
SELECT TOP 10 
    OrderID, 
    PickingCompletedWhen,
    OrderDate,
    ExpectedDeliveryDate
FROM Sales.Orders
WHERE PickingCompletedWhen IS NOT NULL
ORDER BY PickingCompletedWhen DESC;

-- Check total counts
SELECT 
    COUNT(*) AS TotalOrders,
    COUNT(PickingCompletedWhen) AS OrdersWithPickingCompleted,
    COUNT(*) - COUNT(PickingCompletedWhen) AS OrdersWithoutPickingCompleted
FROM Sales.Orders;

-- Main query: Orders with PickingCompletedWhen NOT NULL
SELECT TOP 20
    OrderID, 
    PickingCompletedWhen,
    OrderDate,
    ExpectedDeliveryDate,
    DATEDIFF(day, OrderDate, PickingCompletedWhen) AS DaysToComplete
FROM Sales.Orders
WHERE PickingCompletedWhen IS NOT NULL
ORDER BY PickingCompletedWhen DESC;

### Task 2 — Zero-picked lines from those orders

In [ ]:
-- Task 2: Zero-picked lines from those orders
-- First, let's check the picking quantities
SELECT TOP 10 
    ol.OrderID, 
    ol.OrderLineID, 
    ol.StockItemID, 
    ol.Quantity, 
    ol.PickedQuantity,
    CASE 
        WHEN ol.PickedQuantity = 0 THEN 'Not Picked'
        WHEN ol.PickedQuantity < ol.Quantity THEN 'Partially Picked'
        WHEN ol.PickedQuantity = ol.Quantity THEN 'Fully Picked'
        WHEN ol.PickedQuantity > ol.Quantity THEN 'Over Picked'
    END AS PickingStatus
FROM Sales.OrderLines AS ol
ORDER BY ol.OrderID, ol.OrderLineID;

-- Check picking statistics
SELECT 
    COUNT(*) AS TotalOrderLines,
    SUM(CASE WHEN PickedQuantity = 0 THEN 1 ELSE 0 END) AS ZeroPickedLines,
    SUM(CASE WHEN PickedQuantity < Quantity THEN 1 ELSE 0 END) AS UnderPickedLines,
    SUM(CASE WHEN PickedQuantity = Quantity THEN 1 ELSE 0 END) AS FullyPickedLines
FROM Sales.OrderLines;

-- Main query: Zero-picked lines from completed orders
SELECT 
    o.OrderID, 
    o.PickingCompletedWhen,
    ol.OrderLineID, 
    ol.StockItemID, 
    ol.Quantity, 
    ol.PickedQuantity,
    ol.Quantity - ol.PickedQuantity AS UnpickedQuantity
FROM Sales.Orders AS o
JOIN Sales.OrderLines AS ol ON ol.OrderID = o.OrderID
WHERE o.PickingCompletedWhen IS NOT NULL
  AND ol.PickedQuantity = 0
ORDER BY o.PickingCompletedWhen DESC, o.OrderID, ol.OrderLineID;

### Task 3 — Were these orders invoiced anyway?

In [ ]:
-- Task 3: Were these orders invoiced anyway?
-- MODIFIED: Let's check if orders with zero-picked lines were still invoiced
SELECT TOP 10
    DISTINCT o.OrderID, 
    i.InvoiceID, 
    i.InvoiceDate,
    o.PickingCompletedWhen,
    COUNT(ol.OrderLineID) AS TotalLines,
    SUM(CASE WHEN ol.PickedQuantity = 0 THEN 1 ELSE 0 END) AS ZeroPickLines
FROM Sales.Orders AS o
JOIN Sales.OrderLines AS ol ON ol.OrderID = o.OrderID
LEFT JOIN Sales.Invoices AS i ON i.OrderID = o.OrderID
WHERE o.PickingCompletedWhen IS NOT NULL
  AND ol.PickedQuantity = 0
GROUP BY o.OrderID, i.InvoiceID, i.InvoiceDate, o.PickingCompletedWhen
ORDER BY o.PickingCompletedWhen DESC;


### Task 4 — Zero-pick tally per completed order

In [ ]:
-- Task 4: Zero-pick tally per completed order
-- MODIFIED: Let's see the distribution of zero-pick lines across completed orders
SELECT TOP 10
    o.OrderID, 
    o.PickingCompletedWhen,
    COUNT(ol.OrderLineID) AS TotalLines,
    SUM(CASE WHEN ol.PickedQuantity = 0 THEN 1 ELSE 0 END) AS ZeroPickLines,
    ROUND(CAST(SUM(CASE WHEN ol.PickedQuantity = 0 THEN 1 ELSE 0 END) AS FLOAT) / COUNT(ol.OrderLineID) * 100, 2) AS ZeroPickPercentage
FROM Sales.Orders AS o
JOIN Sales.OrderLines AS ol ON ol.OrderID = o.OrderID
WHERE o.PickingCompletedWhen IS NOT NULL
GROUP BY o.OrderID, o.PickingCompletedWhen
HAVING SUM(CASE WHEN ol.PickedQuantity = 0 THEN 1 ELSE 0 END) > 0
ORDER BY ZeroPickPercentage DESC, o.PickingCompletedWhen DESC;


### Consolidated CTE

In [ ]:
WITH task_1 AS (
  SELECT OrderID, PickingCompletedWhen
  FROM Sales.Orders
  WHERE PickingCompletedWhen IS NOT NULL
),
task_2 AS (
  SELECT ol.OrderID, COUNT(*) AS ZeroPickLines
  FROM Sales.OrderLines AS ol
  JOIN task_1 AS p ON p.OrderID = ol.OrderID
  WHERE ol.PickedQuantity = 0
  GROUP BY ol.OrderID
  HAVING COUNT(*) > 0
),
task_3 AS (
  SELECT i.OrderID, MIN(i.InvoiceDate) AS FirstInvoiceDate
  FROM Sales.Invoices AS i
  GROUP BY i.OrderID
)
SELECT p.OrderID, p.PickingCompletedWhen, z.ZeroPickLines, t3.FirstInvoiceDate
FROM task_1 AS p
JOIN task_2 AS z  ON z.OrderID = p.OrderID
LEFT JOIN task_3 AS t3 ON t3.OrderID = p.OrderID
ORDER BY p.PickingCompletedWhen DESC;

## Case #004 — The Warehouse Wraith
**Logline:** Stock walks out without a paper trail. The cameras caught a shadow; the ledger caught nothing.

### Task 1 — Negative stock movements

In [ ]:
-- Task 1: Negative stock movements
-- First, let's check the stock transaction data
SELECT TOP 10 
    StockItemTransactionID, 
    StockItemID, 
    Quantity, 
    TransactionOccurredWhen,
    TransactionTypeID,
    InvoiceID,
    PurchaseOrderID,
    CustomerID,
    SupplierID
FROM Warehouse.StockItemTransactions
ORDER BY TransactionOccurredWhen DESC;

-- Check quantity distribution
SELECT 
    CASE 
        WHEN Quantity < 0 THEN 'Negative'
        WHEN Quantity = 0 THEN 'Zero'
        WHEN Quantity > 0 THEN 'Positive'
    END AS QuantityType,
    COUNT(*) AS TransactionCount,
    SUM(Quantity) AS TotalQuantity
FROM Warehouse.StockItemTransactions
GROUP BY 
    CASE 
        WHEN Quantity < 0 THEN 'Negative'
        WHEN Quantity = 0 THEN 'Zero'
        WHEN Quantity > 0 THEN 'Positive'
    END;

-- Main query: Negative stock movements
SELECT TOP 20
    StockItemTransactionID, 
    StockItemID, 
    Quantity, 
    TransactionOccurredWhen,
    TransactionTypeID,
    InvoiceID,
    PurchaseOrderID,
    CustomerID,
    SupplierID
FROM Warehouse.StockItemTransactions
WHERE Quantity < 0
ORDER BY TransactionOccurredWhen DESC;

### Task 2 — No linked paperwork (invoice/PO/customer/supplier all NULL)

In [ ]:
-- Task 2: No linked paperwork (invoice/PO/customer/supplier all NULL)
-- Check paperwork linkage statistics
SELECT 
    COUNT(*) AS TotalNegativeTransactions,
    COUNT(InvoiceID) AS WithInvoiceID,
    COUNT(PurchaseOrderID) AS WithPurchaseOrderID,
    COUNT(CustomerID) AS WithCustomerID,
    COUNT(SupplierID) AS WithSupplierID,
    COUNT(*) - COUNT(InvoiceID) - COUNT(PurchaseOrderID) - COUNT(CustomerID) - COUNT(SupplierID) AS WithNoPaperwork
FROM Warehouse.StockItemTransactions
WHERE Quantity < 0;

-- Main query: Negative transactions with no linked paperwork
SELECT 
    StockItemTransactionID,
    StockItemID,
    Quantity,
    TransactionOccurredWhen,
    TransactionTypeID,
    'No Paperwork' AS IssueType
FROM Warehouse.StockItemTransactions
WHERE Quantity < 0
  AND InvoiceID IS NULL
  AND PurchaseOrderID IS NULL
  AND CustomerID IS NULL
  AND SupplierID IS NULL
ORDER BY TransactionOccurredWhen DESC;

### Task 3 — Decorate with item & transaction type names

In [ ]:
SELECT sit.StockItemTransactionID, si.StockItemName, sit.TransactionOccurredWhen,
       sit.Quantity, tt.TransactionTypeName
FROM Warehouse.StockItemTransactions AS sit
JOIN Warehouse.StockItems AS si ON si.StockItemID = sit.StockItemID
LEFT JOIN Application.TransactionTypes AS tt ON tt.TransactionTypeID = sit.TransactionTypeID
WHERE sit.Quantity < 0
  AND sit.InvoiceID IS NULL
  AND sit.PurchaseOrderID IS NULL
  AND sit.CustomerID IS NULL
  AND sit.SupplierID IS NULL;

### Task 4 — Scoreboard: ghosts by item

In [ ]:
SELECT si.StockItemName, COUNT(*) AS GhostMoves, SUM(sit.Quantity) AS NetQty
FROM Warehouse.StockItemTransactions AS sit
JOIN Warehouse.StockItems AS si ON si.StockItemID = sit.StockItemID
WHERE sit.Quantity < 0
  AND sit.InvoiceID IS NULL AND sit.PurchaseOrderID IS NULL
  AND sit.CustomerID IS NULL AND sit.SupplierID IS NULL
GROUP BY si.StockItemName
ORDER BY GhostMoves DESC;

### Consolidated CTE

In [ ]:
WITH task_1 AS (
  SELECT StockItemTransactionID, StockItemID, TransactionOccurredWhen,
         TransactionTypeID, Quantity, InvoiceID, PurchaseOrderID, CustomerID, SupplierID
  FROM Warehouse.StockItemTransactions
  WHERE Quantity < 0
),
task_2 AS (
  SELECT *
  FROM task_1
  WHERE InvoiceID IS NULL
    AND PurchaseOrderID IS NULL
    AND CustomerID IS NULL
    AND SupplierID IS NULL
),
task_3 AS (
  SELECT t2.StockItemTransactionID, si.StockItemName, t2.TransactionOccurredWhen,
         t2.Quantity, tt.TransactionTypeName
  FROM task_2 AS t2
  JOIN Warehouse.StockItems AS si ON si.StockItemID = t2.StockItemID
  LEFT JOIN Application.TransactionTypes AS tt ON tt.TransactionTypeID = t2.TransactionTypeID
)
SELECT *
FROM task_3
ORDER BY TransactionOccurredWhen DESC;

## Case #005 — The Cold-Room Conspiracy
**Logline:** The cooler coughs. The thermometer jumps. Trucks idle after dark.

### Task 1 — Spike days in cold-room readings (MaxT > 6.0°C)

In [ ]:
-- Task 1: Spike days in cold-room readings (MaxT > 6.0°C)
-- First, let's check the cold room temperature data
SELECT TOP 10 
    RecordedWhen,
    Temperature,
    CAST(RecordedWhen AS date) AS RecordDate
FROM Warehouse.ColdRoomTemperatures_Archive
ORDER BY RecordedWhen DESC;

-- Check temperature statistics
SELECT 
    COUNT(*) AS TotalReadings,
    MIN(Temperature) AS MinTemp,
    MAX(Temperature) AS MaxTemp,
    AVG(Temperature) AS AvgTemp,
    COUNT(CASE WHEN Temperature > 6.0 THEN 1 END) AS ReadingsAbove6C
FROM Warehouse.ColdRoomTemperatures_Archive;

-- Main query: Spike days (MaxT > 6.0°C)
SELECT TOP 20
    CAST(RecordedWhen AS date) AS SpikeDate, 
    MAX(Temperature) AS MaxTemperature,
    MIN(Temperature) AS MinTemperature,
    AVG(Temperature) AS AvgTemperature,
    COUNT(*) AS ReadingCount
FROM Warehouse.ColdRoomTemperatures_Archive
GROUP BY CAST(RecordedWhen AS date)
HAVING MAX(Temperature) > 6.0
ORDER BY SpikeDate DESC;

### Task 2 — Chiller volume per day

In [ ]:
-- Task 2: Chiller volume per day
-- First, let's check chiller items data
SELECT TOP 10 
    InvoiceID,
    InvoiceDate,
    TotalChillerItems,
    TotalDryItems
FROM Sales.Invoices
WHERE TotalChillerItems > 0
ORDER BY InvoiceDate DESC;

-- Check chiller statistics
SELECT 
    COUNT(*) AS TotalInvoices,
    SUM(TotalChillerItems) AS TotalChillerItems,
    AVG(TotalChillerItems) AS AvgChillerItemsPerInvoice,
    COUNT(CASE WHEN TotalChillerItems > 0 THEN 1 END) AS InvoicesWithChillerItems
FROM Sales.Invoices;

-- Main query: Chiller volume per day
SELECT 
    InvoiceDate AS ChillerDate, 
    COUNT(*) AS InvoiceCount,
    SUM(TotalChillerItems) AS TotalChillerItems,
    AVG(TotalChillerItems) AS AvgChillerItemsPerInvoice
FROM Sales.Invoices
GROUP BY InvoiceDate
ORDER BY InvoiceDate DESC;

### Task 3 — After-hours deliveries per day

In [ ]:
SELECT CAST(ConfirmedDeliveryTime AS date) AS D, COUNT(*) AS LateDrops
FROM Sales.Invoices
WHERE ConfirmedDeliveryTime IS NOT NULL
  AND (DATEPART(HOUR, ConfirmedDeliveryTime) < 7
    OR DATEPART(HOUR, ConfirmedDeliveryTime) > 21)
GROUP BY CAST(ConfirmedDeliveryTime AS date);

### Task 4 — Join spikes ↔ chiller volume ↔ late drops

In [ ]:
WITH spikes AS (
  SELECT CAST(RecordedWhen AS date) AS D, MAX(Temperature) AS MaxT
  FROM Warehouse.ColdRoomTemperatures_Archive
  GROUP BY CAST(RecordedWhen AS date)
  HAVING MAX(Temperature) > 6.0
)
SELECT s.D, s.MaxT, ISNULL(c.ChillerItems,0) AS ChillerItems, ISNULL(r.LateDrops,0) AS LateDrops
FROM spikes AS s
LEFT JOIN (
  SELECT InvoiceDate AS D, SUM(TotalChillerItems) AS ChillerItems
  FROM Sales.Invoices GROUP BY InvoiceDate
) AS c ON c.D = s.D
LEFT JOIN (
  SELECT CAST(ConfirmedDeliveryTime AS date) AS D, COUNT(*) AS LateDrops
  FROM Sales.Invoices
  WHERE ConfirmedDeliveryTime IS NOT NULL
    AND (DATEPART(HOUR, ConfirmedDeliveryTime) < 7 OR DATEPART(HOUR, ConfirmedDeliveryTime) > 21)
  GROUP BY CAST(ConfirmedDeliveryTime AS date)
) AS r ON r.D = s.D
ORDER BY s.D;

### Consolidated CTE

In [ ]:
WITH task_1 AS (
  SELECT CAST(RecordedWhen AS date) AS D, MAX(Temperature) AS MaxT
  FROM Warehouse.ColdRoomTemperatures_Archive
  GROUP BY CAST(RecordedWhen AS date)
  HAVING MAX(Temperature) > 6.0
),
task_2 AS (
  SELECT InvoiceDate AS D, SUM(TotalChillerItems) AS ChillerItems
  FROM Sales.Invoices GROUP BY InvoiceDate
),
task_3 AS (
  SELECT CAST(ConfirmedDeliveryTime AS date) AS D, COUNT(*) AS LateDrops
  FROM Sales.Invoices
  WHERE ConfirmedDeliveryTime IS NOT NULL
    AND (DATEPART(HOUR, ConfirmedDeliveryTime) < 7
      OR DATEPART(HOUR, ConfirmedDeliveryTime) > 21)
  GROUP BY CAST(ConfirmedDeliveryTime AS date)
)
SELECT s.D AS SpikeDate, s.MaxT, ISNULL(c.ChillerItems,0) AS ChillerItems, ISNULL(r.LateDrops,0) AS LateDeliveries
FROM task_1 AS s
LEFT JOIN task_2 AS c ON c.D = s.D
LEFT JOIN task_3 AS r ON r.D = s.D
ORDER BY s.D;

## Case #006 — The Credit-Hold Caper
**Logline:** Finance freezes the line. Sales thaw it out with a smile.

### Task 1 — List customers on credit hold

In [ ]:
-- Task 1: List customers on credit hold
SELECT 
    COUNT(*) AS TotalCustomers,
    AVG(CreditLimit) AS AvgCreditLimit,
    MAX(CreditLimit) AS MaxCreditLimit,
    MIN(CreditLimit) AS MinCreditLimit
FROM Sales.Customers
WHERE CreditLimit IS NOT NULL;

-- Check credit limit ranges
SELECT 
    CASE 
        WHEN CreditLimit >= 100000 THEN 'Very High (100K+)'
        WHEN CreditLimit >= 50000 THEN 'High (50K-100K)'
        WHEN CreditLimit >= 10000 THEN 'Medium (10K-50K)'
        WHEN CreditLimit >= 1000 THEN 'Low (1K-10K)'
        ELSE 'Very Low (<1K)'
    END AS CreditRange,
    COUNT(*) AS CustomerCount
FROM Sales.Customers
WHERE CreditLimit IS NOT NULL
GROUP BY 
    CASE 
        WHEN CreditLimit >= 100000 THEN 'Very High (100K+)'
        WHEN CreditLimit >= 50000 THEN 'High (50K-100K)'
        WHEN CreditLimit >= 10000 THEN 'Medium (10K-50K)'
        WHEN CreditLimit >= 1000 THEN 'Low (1K-10K)'
        ELSE 'Very Low (<1K)'
    END
ORDER BY CustomerCount DESC;

-- Main query: Customers with highest credit limits
SELECT TOP 20
    CustomerID, 
    CustomerName,
    IsOnCreditHold,
    CreditLimit,
    AccountOpenedDate
FROM Sales.Customers
WHERE CreditLimit IS NOT NULL
ORDER BY CreditLimit DESC;

### Task 2 — Pull their invoices

In [ ]:
-- Task 2: Pull their invoices
SELECT 
    COUNT(*) AS TotalInvoicesForHighCreditCustomers
FROM Sales.Invoices AS i
JOIN Sales.Customers AS c ON c.CustomerID = i.CustomerID
WHERE c.CreditLimit >= 50000;

-- Main query: Invoices for customers with highest credit limits
SELECT TOP 20
    i.InvoiceID, 
    i.InvoiceDate, 
    i.CustomerID,
    c.CustomerName,
    c.CreditLimit,
    c.IsOnCreditHold
FROM Sales.Invoices AS i
JOIN Sales.Customers AS c ON c.CustomerID = i.CustomerID
WHERE c.CreditLimit >= 50000
ORDER BY c.CreditLimit DESC, i.InvoiceDate DESC;

### Task 3 — Compute invoice amounts from lines

In [ ]:
-- Task 3: Compute invoice amounts from lines
-- First, let's check invoice line data
SELECT TOP 10 
    il.InvoiceID,
    il.Quantity,
    il.UnitPrice,
    il.Quantity * ISNULL(il.UnitPrice,0) AS LineAmount
FROM Sales.InvoiceLines AS il
ORDER BY il.InvoiceID;

-- Check amount statistics
SELECT 
    COUNT(*) AS TotalInvoiceLines,
    SUM(il.Quantity * ISNULL(il.UnitPrice,0)) AS TotalAmount,
    AVG(il.Quantity * ISNULL(il.UnitPrice,0)) AS AvgLineAmount,
    MIN(il.Quantity * ISNULL(il.UnitPrice,0)) AS MinLineAmount,
    MAX(il.Quantity * ISNULL(il.UnitPrice,0)) AS MaxLineAmount
FROM Sales.InvoiceLines AS il;

-- Main query: Invoice amounts from lines
SELECT 
    il.InvoiceID, 
    COUNT(*) AS LineCount,
    SUM(il.Quantity * ISNULL(il.UnitPrice,0)) AS TotalAmount,
    AVG(il.Quantity * ISNULL(il.UnitPrice,0)) AS AvgLineAmount
FROM Sales.InvoiceLines AS il
GROUP BY il.InvoiceID
ORDER BY TotalAmount DESC;

### Task 4 — Sort by most recent and largest

In [ ]:
-- Task 4: Sort by most recent and largest
WITH amt AS (
  SELECT il.InvoiceID, SUM(il.Quantity * ISNULL(il.UnitPrice,0)) AS Amount
  FROM Sales.InvoiceLines AS il GROUP BY il.InvoiceID
)
SELECT TOP 20
    i.InvoiceID, 
    i.InvoiceDate, 
    c.CustomerName, 
    c.CreditLimit,
    a.Amount
FROM Sales.Invoices AS i
JOIN Sales.Customers AS c ON c.CustomerID = i.CustomerID
JOIN amt AS a ON a.InvoiceID = i.InvoiceID
WHERE c.CreditLimit >= 50000
ORDER BY c.CreditLimit DESC, i.InvoiceDate DESC, a.Amount DESC;

### Consolidated CTE

In [ ]:
WITH task_1 AS (
  SELECT CustomerID, CustomerName, CreditLimit FROM Sales.Customers 
  WHERE CreditLimit >= 50000
),
task_2 AS (
  SELECT i.InvoiceID, i.InvoiceDate, i.CustomerID
  FROM Sales.Invoices AS i
  JOIN task_1 AS oh ON oh.CustomerID = i.CustomerID
),
task_3 AS (
  SELECT il.InvoiceID, SUM(il.Quantity * ISNULL(il.UnitPrice,0)) AS Amount
  FROM Sales.InvoiceLines AS il
  GROUP BY il.InvoiceID
)
SELECT TOP 20
    t2.InvoiceID, 
    t2.InvoiceDate, 
    oh.CustomerName, 
    oh.CreditLimit, 
    a.Amount
FROM task_2 AS t2
JOIN task_1 AS oh ON oh.CustomerID = t2.CustomerID
JOIN task_3 AS a  ON a.InvoiceID   = t2.InvoiceID
ORDER BY oh.CreditLimit DESC, t2.InvoiceDate DESC;

## Case #007 — The Banking Switcheroo
**Logline:** A supplier’s bank digits dance, and a fat wire follows the beat.

### Task 1 — Treat ValidFrom as the supplier change moment

In [ ]:
-- Task 1: Treat ValidFrom as the supplier change moment
-- First, let's check supplier data
SELECT TOP 10 
    SupplierID, 
    SupplierName, 
    ValidFrom,
    ValidTo,
    CAST(ValidFrom AS date) AS ChangeDate
FROM Purchasing.Suppliers
ORDER BY ValidFrom DESC;

-- Check supplier change statistics
SELECT 
    COUNT(*) AS TotalSuppliers,
    COUNT(ValidFrom) AS SuppliersWithValidFrom,
    MIN(ValidFrom) AS EarliestChange,
    MAX(ValidFrom) AS LatestChange
FROM Purchasing.Suppliers;

-- Main query: Supplier change moments
SELECT TOP 20
    SupplierID, 
    SupplierName, 
    ValidFrom,
    CAST(ValidFrom AS date) AS ChangeDate,
    ValidTo,
    CAST(ValidTo AS date) AS ValidToDate
FROM Purchasing.Suppliers
ORDER BY ValidFrom DESC;

### Task 2 — List supplier transactions with type names

In [ ]:
-- Task 2: List supplier transactions with type names
-- First, let's check transaction types
SELECT 
    TransactionTypeID,
    TransactionTypeName,
    COUNT(*) AS UsageCount
FROM Application.TransactionTypes
GROUP BY TransactionTypeID, TransactionTypeName
ORDER BY TransactionTypeID;

-- Check supplier transaction statistics
SELECT 
    COUNT(*) AS TotalSupplierTransactions,
    COUNT(TransactionTypeID) AS TransactionsWithType,
    SUM(TransactionAmount) AS TotalAmount,
    AVG(TransactionAmount) AS AvgAmount,
    MIN(TransactionAmount) AS MinAmount,
    MAX(TransactionAmount) AS MaxAmount
FROM Purchasing.SupplierTransactions;

-- Main query: Supplier transactions with type names
SELECT 
    st.SupplierTransactionID, 
    st.SupplierID, 
    st.TransactionDate, 
    st.TransactionAmount,
    st.TransactionTypeID,
    tt.TransactionTypeName,
    CASE 
        WHEN st.TransactionAmount >= 10000 THEN 'High Value'
        WHEN st.TransactionAmount >= 1000 THEN 'Medium Value'
        ELSE 'Low Value'
    END AS ValueCategory
FROM Purchasing.SupplierTransactions AS st
LEFT JOIN Application.TransactionTypes AS tt
  ON tt.TransactionTypeID = st.TransactionTypeID
ORDER BY st.TransactionDate DESC, st.TransactionAmount DESC;

### Task 3 — Same-day payments above a threshold

In [ ]:
WITH changes AS (
  SELECT SupplierID, SupplierName, CAST(ValidFrom AS date) AS ChangeDate
  FROM Purchasing.Suppliers
)
SELECT ch.SupplierID, ch.SupplierName, ch.ChangeDate, st.SupplierTransactionID,
       st.TransactionDate, st.TransactionAmount
FROM changes AS ch
JOIN Purchasing.SupplierTransactions AS st
  ON st.SupplierID = ch.SupplierID
 AND st.TransactionDate = ch.ChangeDate
WHERE st.TransactionAmount >= 10000.00;

### Task 4 — Restrict to payment-like transaction types

In [ ]:
WITH changes AS (
  SELECT SupplierID, CAST(ValidFrom AS date) AS ChangeDate
  FROM Purchasing.Suppliers
)
SELECT ch.SupplierID, st.SupplierTransactionID, st.TransactionDate, st.TransactionAmount, tt.TransactionTypeName
FROM changes AS ch
JOIN Purchasing.SupplierTransactions AS st
  ON st.SupplierID = ch.SupplierID AND st.TransactionDate = ch.ChangeDate
LEFT JOIN Application.TransactionTypes AS tt ON tt.TransactionTypeID = st.TransactionTypeID
WHERE st.TransactionAmount >= 10000.00
  AND (tt.TransactionTypeName LIKE '%Payment%' OR tt.TransactionTypeName LIKE '%Receipt%');

### Consolidated CTE

In [ ]:
WITH task_1 AS (
  SELECT SupplierID, SupplierName, CAST(ValidFrom AS date) AS ChangeDate
  FROM Purchasing.Suppliers
),
task_2 AS (
  SELECT st.SupplierTransactionID, st.SupplierID, st.TransactionDate, st.TransactionAmount,
         tt.TransactionTypeName
  FROM Purchasing.SupplierTransactions AS st
  LEFT JOIN Application.TransactionTypes AS tt ON tt.TransactionTypeID = st.TransactionTypeID
),
task_3 AS (
  SELECT ch.SupplierID, ch.SupplierName, ch.ChangeDate,
         t.SupplierTransactionID, t.TransactionDate, t.TransactionAmount, t.TransactionTypeName
  FROM task_1 AS ch
  JOIN task_2 AS t
    ON t.SupplierID = ch.SupplierID
   AND t.TransactionDate = ch.ChangeDate
  WHERE t.TransactionAmount >= 10000.00
)
SELECT *
FROM task_3
WHERE (TransactionTypeName LIKE '%Payment%' OR TransactionTypeName LIKE '%Receipt%')
ORDER BY ChangeDate DESC, TransactionAmount DESC;